In [1]:
# Film Analysis Project - 23rd November 2022 - Time ALlowed: 2 hours.

import pandas as pd

# We have 3 data sets (Ratings, Users, Movies)

# First, load the ratings data set

ratings_df = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python')

# Add column names to ratings data frame

ratings_df.rename(columns = {0: 'User ID', 1: 'Movie ID', 2: 'Rating', 3: 'Time Stamp'}, inplace = True)

# Second, load the users data set

users_df = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python')

# Add column names to users data frame

users_df.rename(columns = {0: 'User ID', 1: 'Gender', 2: 'Age', 3: 'Occupation', 4: 'Zip Code'}, inplace = True)

# Lastly, load the movies data set

movies_df = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python')

# Add column names to movies data frame

movies_df.rename(columns = {0: 'Movie ID', 1: 'Title', 2: 'Genre'}, inplace = True)

# Split the genre in movies into their seperate columns  

Genres_df = movies_df['Genre'].str.split(pat = '|', expand = True)

# As genres only exist in first 3 columns, drop the last two columns that are populated with 'None'

Genres_df = Genres_df.iloc[:, :3]

# Insert the Genre columns into movies data frame

movies_df.insert(loc = 3, column = 'Genre 1', value = Genres_df[0])

movies_df.insert(loc = 4, column = 'Genre 2', value = Genres_df[1])

movies_df.insert(loc = 5, column = 'Genre 3', value = Genres_df[2])

# Drop the Genre column from movies_df

movies_df.drop('Genre', axis = 1, inplace = True)

# Merge the users data frame onto ratings data frame

ratings_df = pd.merge(ratings_df, users_df, on = 'User ID', how = 'left')

# Merge the movies data frame onto the newly created ratings data frame to create our final data frame

ratings_df = pd.merge(ratings_df, movies_df, on = 'Movie ID', how = 'left')

# Display the ratings data frame

ratings_df.head(5)

,User ID,Movie ID,Rating,Time Stamp,Gender,Age,Occupation,Zip Code,Title,Genre 1,Genre 2,Genre 3
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,None,None
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation,Children's,Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical,Romance,None
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,None,None
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation,Children's,Comedy


In [2]:
# Question 1 - Determine the average rating for each movie.

avg_rating = ratings_df.groupby('Title')[['Rating']].mean().round(decimals = 2)

avg_rating.head(10)

,Rating
Title,
"$1,000,000 Duck (1971)",3.03
'Night Mother (1986),3.37
'Til There Was You (1997),2.69
"'burbs, The (1989)",2.91
...And Justice for All (1979),3.71
1-900 (1994),2.50
10 Things I Hate About You (1999),3.42
101 Dalmatians (1961),3.60
101 Dalmatians (1996),3.05


In [3]:
# Question 2 - Do males and females like the same movie? What are the top movies for male and female viewers.

# Create a filtered data set for male viewers

males = ratings_df[ratings_df["Gender"] == "M"]

# Create a filtered data set for female viewers

females = ratings_df[ratings_df["Gender"] == "F"]

# Calculate the average rating for each movie for male viewers

avg_rating_males = males.groupby('Title')[['Rating']].mean().sort_values('Rating', ascending = False).round(decimals = 2)

# Calculate the average rating for each movie for male viewers

avg_rating_females = females.groupby('Title')[['Rating']].mean().sort_values('Rating', ascending = False).round(decimals = 2)

# Display the average ratings sorted by rating for males and females. We see that the two genders do NOT have an overlap 
# in their top rated films.

print(avg_rating_males)
print(avg_rating_females)

                                                    Rating
Title                                                     
Schlafes Bruder (Brother of Sleep) (1995)              5.0
Small Wonders (1996)                                   5.0
Gate of Heavenly Peace, The (1995)                     5.0
Baby, The (1973)                                       5.0
Ulysses (Ulisse) (1954)                                5.0
...                                                    ...
White Boys (1999)                                      1.0
Waltzes from Vienna (1933)                             1.0
Blood Spattered Bride, The (La Novia Ensangrent...     1.0
Kestrel's Eye (Falkens öga) (1998)                     1.0
Mutters Courage (1995)                                 1.0

[3671 rows x 1 columns]
                                                    Rating
Title                                                     
Clean Slate (Coup de Torchon) (1981)                   5.0
Ballad of Narayama, The (Naraya

In [4]:
# Question 3 - Which movies the female and male viewers disagree with their rating the most?

# Calculate the average ratings for each film grouped by Males and Females

ratings_mf = ratings_df.groupby(['Title', 'Gender']).agg({'Rating': ['mean']}).round(decimals = 2)

ratings_mf.columns = ['Avg. Rating']

ratings_mf = ratings_mf.reset_index()

# Create a filtered data set for female viewers

female = ratings_mf[ratings_mf["Gender"] == "F"]

# Create a filtered data set for female viewers

male = ratings_mf[ratings_mf["Gender"] == "M"]

# Merge the average ratings for male viewers with the average ratings for female viewers

female = pd.merge(female, male, on = 'Title', how = 'left')

# Rename the column names 

female.rename(columns = {'Gender_x': 'Gender_F', 'Avg. Rating_x': 'Avg. Rating_F', 'Gender_y': 'Gender_M', 
                            'Avg. Rating_y': 'Avg. Rating_M'}, inplace = True)

# Calculate the difference between females and males average ratings for each movie

female['Difference'] = (female['Avg. Rating_F'] - female['Avg. Rating_M']).abs()

# Sort the values 

female = female.sort_values('Difference', ascending = False).round(decimals = 2)

# Display the data frame that shows the films that male and female viewers disagree on the most

female.head(10)

,Title,Gender_F,Avg. Rating_F,Gender_M,Avg. Rating_M,Difference
3163,Tigrero: A Film That Was Never Made (1994),F,1.00,M,4.33,3.33
1643,"James Dean Story, The (1957)",F,4.00,M,1.00,3.00
731,Country Life (1994),F,5.00,M,2.00,3.00
2938,"Spiders, The (Die Spinnen, 1. Teil: Der Golden...",F,4.00,M,1.00,3.00
2198,"Neon Bible, The (1995)",F,1.00,M,4.00,3.00
991,"Enfer, L' (1994)",F,1.00,M,3.75,2.75
242,Babyfever (1994),F,3.67,M,1.00,2.67
2954,Stalingrad (1993),F,1.00,M,3.59,2.59
3439,"Woman of Paris, A (1923)",F,5.00,M,2.43,2.57
678,Cobra (1925),F,4.00,M,1.50,2.50


In [5]:
# Question 4 - Which movie genres are rated the highest?

# We have 3 columns for genre. First, I will create the average rating for each genre

# Average Rating for Genre 1

avg_rating_genre1 = ratings_df.groupby('Genre 1')[['Rating']].mean().round(decimals = 2)

# Rename the column name

avg_rating_genre1.rename(columns = {'Rating': 'Rating 1'}, inplace = True)

# Average Rating for Genre 2

avg_rating_genre2 = ratings_df.groupby('Genre 2')[['Rating']].mean().round(decimals = 2)

# Rename the column name

avg_rating_genre2.rename(columns = {'Rating': 'Rating 2'}, inplace = True)

# Average Rating for Genre 3

avg_rating_genre3 = ratings_df.groupby('Genre 3')[['Rating']].mean().round(decimals = 2)

# Rename the column name

avg_rating_genre3.rename(columns = {'Rating': 'Rating 3'}, inplace = True)

# Join the 3 average ratings data frames together 

avg_rating_genre = pd.concat([avg_rating_genre1, avg_rating_genre2,avg_rating_genre3], axis = 1)

# Calculate the overall average ratings

avg_rating_genre["Avg. Rating"] = avg_rating_genre.mean(axis=1, skipna= True).round(decimals = 2)

# Display the average rating for each genre sorted by average rating

avg_rating_genre.sort_values('Avg. Rating', ascending = False)

,Rating 1,Rating 2,Rating 3,Avg. Rating
Film-Noir,4.17,4.11,3.74,4.01
Documentary,3.94,3.92,NaN,3.93
War,3.89,3.94,3.89,3.91
Drama,3.77,3.74,3.90,3.80
Mystery,3.79,3.68,3.56,3.68
Western,3.85,3.72,3.48,3.68
Musical,3.86,3.53,3.61,3.67
Crime,3.88,3.65,3.37,3.63
Animation,3.73,3.41,3.63,3.59
Romance,3.49,3.60,3.61,3.57
